# 自查询

自查询检索器，顾名思义，是一种能够查询自身的检索器。具体来说，给定任何自然语言查询，检索器使用查询构造 LLM 链来编写结构化查询，然后将该结构化查询应用于其底层 VectorStore。这允许检索器不仅使用用户输入的查询与存储的文档的内容进行语义相似性比较，而且还从对存储的文档的元数据的用户查询中提取过滤器并执行这些过滤器。

![image](../images/self_querying.png)


出于演示目的，我们将使用 Chroma 矢量存储。我们创建了一个小型演示文档集，其中包含电影摘要。

注意：自查询检索器要求您安装 lark 软件包。

```
pip install --upgrade --quiet  lark chromadb
```

In [2]:
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document


docs = [
    Document(
        page_content="一群科学家带回恐龙爆发了混乱",
        metadata={"years": 1993, "rating": 7.5, "genre": "科幻小说"}
    ),
    Document(
        page_content="故事发生在1920年北洋年间中国南方，马邦德花钱买官，购得“萨南康省”的县长一职，做“马拉的火车”赴任途中遭马匪张麻子一行人伏击",
        metadata={"years": 2010, "director": "姜文", "rating": 8.5}
    ),
    Document(
        page_content="话说孙悟空护送唐三藏前往西天取经，半路和牛魔王合谋杀害唐三藏， 并偷走了紫霞仙子持有的月光宝盒。观音闻讯赶到，欲除掉孙悟空以免危害苍生。唐三藏慈悲为怀，不忍下手，观音遂赐唐三藏紧箍咒，欲借孙悟空之手除掉牛魔王。",
        metadata={"years": 1994, "derector": "刘振伟", "rating": 8.6}
    ),
    Document(
        page_content="故事背景设定在2075年，讲述了太阳即将毁灭，毁灭之后的太阳系已经不适合人类生存，面对绝境，人类将开启“流浪地球”计划，试图带着地球一起逃离太阳系，寻找新的家园。",
        metadata={"years": 2019, "director": "郭帆", "rating": 8.3}
    ),
    Document(
        page_content="该片讲述了耿浩和好哥们郝义一场荒诞而有趣的“寻爱之旅”。该片采用双线叙事的手法，以耿浩和康晓丽婚姻破裂为叙事的起点，在郝义携耿浩前往局组送道具途中“寻爱”的故事中，穿插着昔日康晓丽孤身前往大理并于耿浩相遇相爱的故事。",
        metadata={"years": 2014, "genre": "喜剧"}
    )
]

In [6]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="/home/libing/kk_LLMs/bge-large-zh-v1.5")

vectorstore = Chroma.from_documents(docs, embeddings)


## 创建自己的自查询检索器


现在我们可以实例化我们的检索器。为此，我们需要预先提供一些有关我们的文档支持的元数据字段以及文档内容的简短描述。

In [13]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

metadata_field_info = [
    AttributeInfo(
        name="years",
        description="电影上映年份",
        type="integer"
    ),
    AttributeInfo(
        name="director",
        description="电影导演",
        type="string"
    ),
    AttributeInfo(
        name="rating",
        description="电影评分 1-10之间",
        type="float"
    ),
    AttributeInfo(
        name="genre",
        description='电影类型。 ["科幻小说","喜剧","剧情片"，"惊悚片","爱情片","动作片","动画片"]之一',
        type="string"
    )
]

document_content_description = "电影摘要"


In [24]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("ZHIPUAI_API_KEY")
base_url = os.getenv("ZHIPUAI_API_BASE")

llm = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.5, max_tokens=8192, model="glm-4-plus")


In [25]:
retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    document_contents=document_content_description,
    metadata_field_info=metadata_field_info,
    enable_limit=True,
    search_kwargs={"k": 2}
)


In [22]:
retriever.invoke("请推荐一部评分8.0以上的电影")

[Document(metadata={'derector': '姜文', 'rating': 8.5, 'years': 2010}, page_content='故事发生在1920年北洋年间中国南方，马邦德花钱买官，购得“萨南康省”的县长一职，做“马拉的火车”赴任途中遭马匪张麻子一行人伏击')]

In [27]:
retriever.invoke("帮我推荐1个关于喜剧的电影")

[Document(metadata={'genre': '喜剧', 'years': 2014}, page_content='该片讲述了耿浩和好哥们郝义一场荒诞而有趣的“寻爱之旅”。该片采用双线叙事的手法，以耿浩和康晓丽婚姻破裂为叙事的起点，在郝义携耿浩前往局组送道具途中“寻爱”的故事中，穿插着昔日康晓丽孤身前往大理并于耿浩相遇相爱的故事。')]